# 1. Carga de datos desde Kedro

En este notebook ejecutamos clustering usando los datos limpios del pipeline de Kedro: `diabetes_cleaned`.

In [ ]:
# ==== Inicializar Kedro correctamente dentro del notebook ====

import pandas as pd
from pathlib import Path

from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession

# Detecta automáticamente el proyecto (2 niveles arriba del notebook)
project_path = Path.cwd().resolve().parents[1]

# Carga la metadata real del proyecto
metadata = bootstrap_project(project_path)

# Crear sesión Kedro
session = KedroSession.create(
    package_name=metadata.package_name,
    project_path=project_path,
)

# Cargar contexto
context = session.load_context()

# Acceder al catálogo
catalog = context.catalog

print("Kedro cargado correctamente ✔️")
print("Datasets disponibles en el catálogo:")
print(catalog.list())


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 project_path = Path.cwd().resolve().parents[1]                                              │
│   11                                                                                             │
│   12 # Carga la metadata real del proyecto                                                       │
│ ❱ 13 metadata = bootstrap_project(project_path)                                                  │
│   14                                                                                             │
│   15 # Crear sesión Kedro                                                                        │
│   16 session = KedroSession.create(                                                              │
│                                                                                                  │
│ c:\Users\goku8\OneDrive\Escritorio\MCHL -                                                        │
│ copiaaaaa\MLY0100-Parcial-1\.venv\Lib\site-packages\kedro\framework\startup.py:154 in            │
│ bootstrap_project                                                                                │
│                                                                                                  │
│   151 │   """                                                                                    │
│   152 │                                                                                          │
│   153 │   project_path = Path(project_path).expanduser().resolve()                               │
│ ❱ 154 │   metadata = _get_project_metadata(project_path)                                         │
│   155 │   _add_src_to_path(metadata.source_dir, project_path)                                    │
│   156 │   configure_project(metadata.package_name)                                               │
│   157 │   return metadata                                                                        │
│                                                                                                  │
│ c:\Users\goku8\OneDrive\Escritorio\MCHL -                                                        │
│ copiaaaaa\MLY0100-Parcial-1\.venv\Lib\site-packages\kedro\framework\startup.py:59 in             │
│ _get_project_metadata                                                                            │
│                                                                                                  │
│    56 │   pyproject_toml = project_path / _PYPROJECT                                             │
│    57 │                                                                                          │
│    58 │   if not pyproject_toml.is_file():                                                       │
│ ❱  59 │   │   raise RuntimeError(                                                                │
│    60 │   │   │   f"Could not find the project configuration file '{_PYPROJECT}' in {project_p   │
│    61 │   │   │   f"If you have created your project with Kedro "                                │
│    62 │   │   │   f"version <0.17.0, make sure to update your project template. "                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Could not find the project configuration file 'pyproject.toml' in 
C:\Users\goku8\OneDrive\Escritorio\MCHL - copiaaaaa\MLY0100-Parcial-1. If you have created your project with Kedro 
version <0.17.0, make sure to update your project template. See 
https://github.com/kedro-org/kedro/blob/main/RELEASE.md#migration-guide-from-kedro-016-to-kedro-0170 for how to 
migrate your Kedro project.

# 2. Selección de variables para clustering
Se eliminan columnas que sean categóricas o irrelevantes para K-Means.

In [ ]:
# Quitamos columnas no numéricas
X = df.select_dtypes(include=['float64', 'int64'])
X.head()

# 3. Escalamiento de datos
K-Means requiere que los datos estén escalados.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:5]

# 4. Aplicación de K-Means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

df['cluster'] = clusters
df[['cluster']].value_counts()

# 5. Visualización PCA para reducir a 2D

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
components = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,5))
plt.scatter(components[:,0], components[:,1], c=clusters, cmap='viridis', s=10)
plt.title("Clustering K-Means con PCA (2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# 6. Guardar resultados (opcional)

In [ ]:
df.to_csv("diabetes_clusters_local.csv", index=False)
df.head()